In [1]:
import pulp
import numpy as np

# Définissez vos variables et constants
N = 5  # Nombre d'actifs disponibles
K = 3  # Nombre maximum d'actifs à inclure

# Générez les rendements moyens espérés et la matrice de covariance (à titre d'exemple)
mu = np.random.rand(N)

# Générez une matrice de covariance symétrique (à titre d'exemple)
covariance_matrix = np.random.rand(N, N)
covariance_matrix = (covariance_matrix + covariance_matrix.T) / 2  # Assurez-vous qu'elle soit symétrique

# Créez le problème PULP
problem = pulp.LpProblem("Portfolio_Optimization", pulp.LpMinimize)

# Définissez les variables de décision (les poids des actifs)
x = pulp.LpVariable.dicts("Weight", range(N), lowBound=0, upBound=1, cat=pulp.LpContinuous)

# Ajoutez la contrainte de budget
budget_constraint = pulp.lpSum(mu[i] * x[i] for i in range(N)) == 1

# Ajoutez la contrainte de somme des poids égale à 1
sum_weights_constraint = pulp.lpSum(x[i] for i in range(N)) == 1

# Ajoutez la contrainte de cardinalité maximale (nombre maximum d'actifs inclus)
cardinality_constraint = pulp.lpSum(x[i] for i in range(N)) <= K



# Ajoutez les contraintes au problème
problem += budget_constraint
problem += sum_weights_constraint
problem += cardinality_constraint







# Créez des variables auxiliaires pour les produits xi * xj
aux_vars = {}
for i in range(N):
    for j in range(N):
        aux_vars[(i, j)] = pulp.LpVariable(f'aux_{i}_{j}', lowBound=0, upBound=1, cat=pulp.LpContinuous)

# Ajoutez les contraintes pour les variables auxiliaires
for i in range(N):
    for j in range(N):
        problem += aux_vars[(i, j)] >= x[i] + x[j] - 1
        problem += aux_vars[(i, j)] <= x[i]
        problem += aux_vars[(i, j)] <= x[j]
        
        
# Calculer la variance du portefeuille en utilisant les variables auxiliaires
variance = pulp.lpSum(covariance_matrix[i][j] * aux_vars[(i, j)] for i in range(N) for j in range(N))

# Ajoutez l'objectif de minimisation de la variance
objective = variance
problem += objective

# Résolvez le problème
problem.solve()

# Affichez les résultats
optimal_weights = [x[i].varValue for i in range(N)]
print("Optimal Portfolio Weights:", optimal_weights)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dominicprenovost/anaconda3/envs/HECFinance/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/4d3cc921fd8341599de0984bb4ce901f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/4d3cc921fd8341599de0984bb4ce901f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 83 COLUMNS
At line 294 RHS
At line 373 BOUNDS
At line 404 ENDATA
Problem MODEL has 78 rows, 30 columns and 185 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 72 (-6) rows, 30 (0) columns and 170 (-15) elements
0  Obj 0 Primal inf 13.862605 (2)
9  Obj 4.8394611 Primal inf 1794.4311 (19)
Primal infeasible - objective value 4.8394611
Presolved problem not optimal, resolve after postsolve
After Postsolve, objective 4.8394611, infeasib